# numerical differentiation examples

demonstration of finite difference methods and Richardson extrapolation

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from numeric_integrator import differentiate

print("library loaded successfully")

## example 1: basic differentiation

compute $\frac{d}{dx}\sin(x)$ at $x=\pi/4$, exact: $\cos(\pi/4) = \frac{\sqrt{2}}{2}$

In [ ]:
f = np.sin
x0 = np.pi / 4
exact = np.cos(x0)

# compare methods
methods = ["forward", "backward", "central", "richardson"]
h = 0.001

print(f"computing derivative at x={x0:.4f}\n")
for method in methods:
    result = differentiate(f, x0, method=method, h=h)
    error = abs(result.value - exact)
    print(f"{method:10s}: value={result.value:.10f}, error={error:.2e}")

print(f"\nexact:      {exact:.10f}")

## example 2: step size analysis

study how error varies with step size h

In [ ]:
f = lambda x: x**3
x0 = 2.0
exact = 3 * x0**2  # derivative of x^3 is 3x^2

h_values = np.logspace(-8, -1, 30)
errors_forward = []
errors_central = []
errors_richardson = []

for h in h_values:
    result_fwd = differentiate(f, x0, method="forward", h=h)
    result_cen = differentiate(f, x0, method="central", h=h)
    result_ric = differentiate(f, x0, method="richardson", h=h, n=3)
    
    errors_forward.append(abs(result_fwd.value - exact))
    errors_central.append(abs(result_cen.value - exact))
    errors_richardson.append(abs(result_ric.value - exact))

# plot
plt.figure(figsize=(10, 6))
plt.loglog(h_values, errors_forward, 'o-', label='forward O(h)', linewidth=2)
plt.loglog(h_values, errors_central, 's-', label='central O(h²)', linewidth=2)
plt.loglog(h_values, errors_richardson, '^-', label='richardson', linewidth=2)
plt.loglog(h_values, h_values, '--', label='O(h)', alpha=0.3)
plt.loglog(h_values, h_values**2, '--', label='O(h²)', alpha=0.3)
plt.xlabel('step size (h)', fontsize=12)
plt.ylabel('absolute error', fontsize=12)
plt.title('differentiation error vs step size', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("note: for very small h, roundoff error dominates")

## example 3: Richardson extrapolation demonstration

show how Richardson improves accuracy

In [ ]:
f = lambda x: np.exp(x) * np.sin(x)
x0 = 1.0
exact = np.exp(x0) * (np.sin(x0) + np.cos(x0))

h = 0.1
print(f"computing d/dx[exp(x)sin(x)] at x={x0}\n")

# different Richardson orders
for n in [1, 2, 3, 4, 5]:
    result = differentiate(f, x0, method="richardson", h=h, n=n)
    error = abs(result.value - exact)
    print(f"richardson n={n}: value={result.value:.12f}, error={error:.2e}, evals={result.n_evaluations}")

print(f"\nexact:          {exact:.12f}")
print(f"\nhigher n improves accuracy but requires more evaluations")

## example 4: second derivative

compute $\frac{d^2}{dx^2}\sin(x) = -\sin(x)$

In [ ]:
from numeric_integrator.differentiators import second_derivative

f = np.sin
x_values = np.linspace(0, 2*np.pi, 10)

plt.figure(figsize=(10, 6))

# compute second derivatives
second_derivs = []
for x in x_values:
    result = second_derivative(f, x, h=0.001)
    second_derivs.append(result.value)

# plot
x_fine = np.linspace(0, 2*np.pi, 200)
plt.plot(x_fine, np.sin(x_fine), 'b-', linewidth=2, label='f(x) = sin(x)', alpha=0.7)
plt.plot(x_fine, -np.sin(x_fine), 'r-', linewidth=2, label="f''(x) = -sin(x) (exact)", alpha=0.7)
plt.plot(x_values, second_derivs, 'ro', markersize=8, label="f''(x) (numerical)")
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('second derivative verification', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

# compute errors
errors = [abs(num + np.sin(x)) for num, x in zip(second_derivs, x_values)]
print(f"average error: {np.mean(errors):.2e}")
print(f"max error:     {np.max(errors):.2e}")

## example 5: gradient computation

compute gradient of multivariable function

In [ ]:
from numeric_integrator.differentiators import derivative_vector

# function: f(x,y) = x^2 + y^2
# gradient: [2x, 2y]
def f(v):
    x, y = v
    return x**2 + y**2

# compute gradient at several points
points = [(1, 1), (2, 0), (0, 3), (-1, 2)]

print("gradient of f(x,y) = x² + y²\n")
for point in points:
    result = derivative_vector(f, np.array(point), h=0.001)
    exact_grad = 2 * np.array(point)
    error = np.linalg.norm(result.value - exact_grad)
    print(f"point {point}:")
    print(f"  numerical: [{result.value[0]:.6f}, {result.value[1]:.6f}]")
    print(f"  exact:     [{exact_grad[0]:.6f}, {exact_grad[1]:.6f}]")
    print(f"  error:     {error:.2e}\n")

## example 6: optimal step size

find step size that balances truncation and roundoff error

In [ ]:
from numeric_integrator.utils import optimal_step_size

f = lambda x: np.exp(x)
x0 = 1.0

# optimal step for different orders
print("optimal step sizes:\n")
for order in [1, 2, 4]:
    h_opt = optimal_step_size(f, x0, order=order)
    print(f"order {order}: h_opt = {h_opt:.2e}")
    
    # test with optimal h
    if order == 1:
        result = differentiate(f, x0, method="forward", h=h_opt)
    else:
        result = differentiate(f, x0, method="central", h=h_opt)
    
    exact = np.exp(x0)
    error = abs(result.value - exact)
    print(f"  error with h_opt: {error:.2e}\n")